# Mise en base de l'arborescence

## Documentation

### Liste des tables

A chaque type de fichiers mxl correspond une table :

#### Table article

A chaque ligne correspond un fichier LEGIARTIxxxxxxxxxxxx.xml.

Les colonnes base_origine, categorie, cid et id\_ localisent le fichier dans l'arborescence plate.

Les colonnes valide et valide\_\* caractérisent la validité du fichier.


#### Table struct

A chaque ligne correspond un fichier LEGITEXTxxxxxxxxxxxx.xml ou JORFTEXTxxxxxxxxxxxx.xml d'un dossier struct.

Les colonnes base_origine, categorie, cid et id\_ localisent le fichier dans l'arborescence plate.

Les colonnes valide et valide\_\* caractérisent la validité du fichier.


#### Table version

A chaque ligne correspond un fichier LEGITEXTxxxxxxxxxxxx.xml ou JORFTEXTxxxxxxxxxxxx.xml d'un dossier version.

Les colonnes base_origine, categorie, cid et id\_ localisent le fichier dans l'arborescence plate.

Les colonnes valide et valide\_\* caractérisent la validité du fichier.


#### Table texte

A chaque ligne correspond un fichier JORFTEXTxxxxxxxxxxxx.xml de la base JORF ou un dossier LEGITEXTxxxxxxxxxxxx ou JORFTEXTxxxxxxxxxxxx de la base LEGI.

Les colonnes base_origine, categorie, cid localisent le fichier dans l'arborescence plate.

Les colonnes valide et valide\_\* caractérisent la validité du fichier.


#### Table section_ta

A chaque ligne correspond un fichier LEGISCTAxxxxxxxxxxxx.xml.

Les colonnes base_origine, categorie, cid et id\_ localisent le fichier dans l'arborescence plate.

Les colonnes valide et valide\_\* caractérisent la validité du fichier.


#### Table conteneur

A chaque ligne correspond un fichier LEGICONTxxxxxxxxxxxx.xml du JORF.

La colonne id\_ localise le fichier.

Les colonnes valide et valide\_\* caractérisent la validité du fichier.


### Liste des champs

#### base_origine

JORF ou LEGI

#### categorie

* code_en_vigueur_LEGI
* code_non_vigueur_LEGI
* TNC_en_vigueur_JORF
* TNC_en_vigueur_LEGI
* TNC_non_vigueur_JORF 
* TNC_non_vigueur_LEGI

#### cid

Pour LEGI, correspond au 2e niveau dans l'arborescence plate.

Pour JORF, correspond au nom du fichier.

#### id_

Correspond au nom des fichiers dans l'arborescence plate. Un underscore est ajouté à la fin car le mot "id" est un mot réservé pour certains langages de programmation, dont python.

## Mode opératoire

* Créer l'utilisateur et la base de données
```
# sudo -i -u postgres
$ createuser --interactive
$ createdb loi
$ psql
# grant all privileges on database loi to loi;
# alter user loi with password 'baba';
```

* Ajouter la ligne suivante à /etc/postgresql/9.6/main/pg_hba.conf : `local loi loi md5`
* Redémarrer postgresql : `service postgresql restart`
* Pour se connecter avec psql :
```
$ psql -U loi loi
```

* Créer les tables :
```
create table article (base_origine varchar(21), categorie varchar(21),
    cid varchar(21), id_ varchar(21), valide boolean);
create table struct (base_origine varchar(21), categorie varchar(21),
    cid varchar(21), id_ varchar(21), valide boolean);
create table version (base_origine varchar(21), categorie varchar(21),
    cid varchar(21), id_ varchar(21), valide boolean);
create table texte (base_origine varchar(21), categorie varchar(21),
    cid varchar(21), valide boolean);
create table section_ta (base_origine varchar(21), categorie varchar(21),
    cid varchar(21), id_ varchar(21), valide boolean);
create table conteneur (id_ varchar(21), valide boolean);
```

* Créer les index :
```
create index on article (cid);
create index on article (id_);
create index on struct (cid);
create index on struct (id_);
create index on version (cid);
create index on version (id_);
create index on texte (cid);
create index on section_ta (cid);
create index on section_ta (id_);
create index on conteneur (id_);
```


In [1]:
import os
import re

import psycopg2

In [18]:
connection = psycopg2.connect(dbname='loi', user='loi', password='baba')
curseur = connection.cursor()

In [2]:
racine_legi = '/home/michel/legi_plat/'
racine_jorf = '/home/michel/jorf_plat/'

In [4]:
def trouve_enfants(dossier):
    enfants = os.listdir(dossier)
    est_fichier_liste = [os.path.isfile(os.path.join(dossier, f)) for f in enfants]
    enfants_fichier = [enfants[i] for i in range(len(enfants)) if est_fichier_liste[i]]
    enfants_dossier = [enfants[i] for i in range(len(enfants)) if not est_fichier_liste[i]]
    
    return enfants_fichier, enfants_dossier

## Traite la base LEGI

In [5]:
def traite_struct(racine, categorie, cid):
    dossier = os.path.join(racine, categorie, cid, 'struct')
    enfants_fichier, enfants_dossier = trouve_enfants(dossier)
    
    assert len(enfants_dossier) == 0
    
    nom_fichier_attendu = '^LEGITEXT\d{12}\.xml$'
    for enfant_fichier in enfants_fichier:
        assert re.match(nom_fichier_attendu, enfant_fichier)
        
        id_ = enfant_fichier[:20]
        curseur.execute("insert into struct (base_origine, categorie, cid, id_, valide) values ('LEGI', %s, %s, %s, True)", (categorie, cid, id_,))
        connection.commit()

In [6]:
def traite_version(racine, categorie, cid):
    dossier = os.path.join(racine, categorie, cid, 'version')
    enfants_fichier, enfants_dossier = trouve_enfants(dossier)
    
    assert len(enfants_dossier) == 0
    
    nom_fichier_attendu = '^LEGITEXT\d{12}\.xml$'
    for enfant_fichier in enfants_fichier:
        assert re.match(nom_fichier_attendu, enfant_fichier)
        
        id_ = enfant_fichier[:20]
        curseur.execute("insert into version (base_origine, categorie, cid, id_, valide) values ('LEGI', %s, %s, %s, True)",
                        (categorie, cid, id_,))
        connection.commit()

In [14]:
def traite_cid(racine, categorie, cid):
    dossier = os.path.join(racine, categorie, cid)
    enfants_fichier, enfants_dossier = trouve_enfants(dossier)

    assert set(enfants_dossier) == set([
        'struct',
        'version',
    ])
    traite_struct(racine, categorie, cid)
    traite_version(racine, categorie, cid)
    
    nom_fichier_article = '^LEGIARTI\d{12}\.xml$'
    nom_fichier_section_ta = '^LEGISCTA\d{12}\.xml$'
    for enfant_fichier in enfants_fichier:
        id_ = enfant_fichier[:20]
        if re.match(nom_fichier_article, enfant_fichier):
            curseur.execute("insert into article (base_origine, categorie, cid, id_, valide) values ('LEGI', %s, %s, %s, True)",
                            (categorie, cid, id_,))
            connection.commit()
        elif re.match(nom_fichier_section_ta, enfant_fichier):
            curseur.execute("insert into section_ta (base_origine, categorie, cid, id_, valide) values ('LEGI', %s, %s, %s, True)",
                            (categorie, cid, id_,))
            connection.commit()
        else:
            raise ValueError(cid)
            
    curseur.execute("insert into texte (base_origine, categorie, cid, valide) values ('LEGI', %s, %s, True)",
                    (categorie, cid,))
    connection.commit()



In [15]:
def traite_categorie(racine, categorie):
    dossier = os.path.join(racine, categorie)
    enfants_fichier, enfants_dossier = trouve_enfants(dossier)
    
    assert len(enfants_fichier) == 0
    
    nom_dossier_attendu = '^' + categorie[-4:] + 'TEXT\d{12}$'
    for enfant_dossier in enfants_dossier:
        assert re.match(nom_dossier_attendu, enfant_dossier)
        
        traite_cid(racine, categorie, enfant_dossier)
    

In [16]:
def traite_legi(racine):
    enfants_fichier, enfants_dossier = trouve_enfants(racine)
    
    assert len(enfants_fichier) == 0
    assert set(enfants_dossier) == set([
            'code_en_vigueur_LEGI',
            'code_non_vigueur_LEGI',
            'TNC_en_vigueur_JORF',
            'TNC_en_vigueur_LEGI',
            'TNC_non_vigueur_JORF',
            'TNC_non_vigueur_LEGI',
        ])
        
    for enfant_dossier in enfants_dossier:
        traite_categorie(racine, enfant_dossier)

In [19]:
traite_legi(racine_legi)

In [20]:
curseur.close()
connection.close()

A l'issue de cette étape, les tables contiennent les nombres suivants de lignes :
* texte : 97926
* article : 1247771
* section_ta : 177124
* struct : 99480
* version : 99944

## Traite la base JORF

In [21]:
connection = psycopg2.connect(dbname='loi', user='loi', password='baba')
curseur = connection.cursor()

In [24]:
enfants_fichier, enfants_dossier = trouve_enfants(racine_jorf)

assert len(enfants_fichier) == 0
assert set(enfants_dossier) == set([
    'article',
    'conteneur',
    'section_ta',
    'texte',
])

In [25]:
# article

enfants_fichier, enfants_dossier = trouve_enfants(os.path.join(racine_jorf, 'article'))

assert len(enfants_dossier) == 0

nom_fichier_article = '^JORFARTI\d{12}\.xml$'

for enfant_fichier in enfants_fichier:
    assert re.match(nom_fichier_article, enfant_fichier)
    id_ = enfant_fichier[:20]
    curseur.execute("insert into article (base_origine, id_, valide) values ('JORF', %s, True)",
                    (id_,))
    connection.commit()


In [29]:
# conteneur

enfants_fichier, enfants_dossier = trouve_enfants(os.path.join(racine_jorf, 'conteneur'))

assert len(enfants_dossier) == 0

nom_fichier_conteneur = '^JORFCONT\d{12}\.xml$'

for enfant_fichier in enfants_fichier:
    assert re.match(nom_fichier_conteneur, enfant_fichier)
    id_ = enfant_fichier[:20]
    curseur.execute("insert into conteneur (id_, valide) values (%s, True)",
                    (id_,))
    connection.commit()
      

In [30]:
# section_ta

enfants_fichier, enfants_dossier = trouve_enfants(os.path.join(racine_jorf, 'section_ta'))

assert len(enfants_dossier) == 0

nom_fichier_section_ta = '^JORFSCTA\d{12}\.xml$'

for enfant_fichier in enfants_fichier:
    assert re.match(nom_fichier_section_ta, enfant_fichier)
    id_ = enfant_fichier[:20]
    curseur.execute("insert into section_ta (base_origine, id_, valide) values ('JORF', %s, True)",
                    (id_,))
    connection.commit()


In [31]:
# texte

enfants_fichier, enfants_dossier = trouve_enfants(os.path.join(racine_jorf, 'texte'))

assert len(enfants_fichier) == 0
assert set(enfants_dossier) == set([
    'struct',
    'version',
])


In [32]:
# struct

enfants_fichier, enfants_dossier = trouve_enfants(os.path.join(racine_jorf, 'texte/struct'))

assert len(enfants_dossier) == 0

nom_fichier_struct = '^JORFTEXT\d{12}\.xml$'

for enfant_fichier in enfants_fichier:
    assert re.match(nom_fichier_struct, enfant_fichier)
    id_ = enfant_fichier[:20]
    curseur.execute("insert into struct (base_origine, id_, valide) values ('JORF', %s, True)",
                    (id_,))
    connection.commit()


In [ ]:
# version

enfants_fichier, enfants_dossier = trouve_enfants(os.path.join(racine_jorf, 'texte/version'))

assert len(enfants_dossier) == 0

nom_fichier_version = '^JORFTEXT\d{12}\.xml$'

for enfant_fichier in enfants_fichier:
    assert re.match(nom_fichier_version, enfant_fichier)
    id_ = enfant_fichier[:20]
    curseur.execute("insert into version (base_origine, id_, valide) values ('JORF', %s, True)",
                    (id_,))
    connection.commit()


In [33]:
curseur.close()
connection.close()

A l'issue de cette étape, les tables contiennent les nombres suivants de lignes :
* texte : 97926 + 0 = 97926
* article : 1247771 + 1959338 = 3207109
* section_ta : 177124 + 89092 = 266216
* struct : 99480 + 961006 = 1060486
* version : 99944 + 961006 = 1060950
* conteneur : 0 + 28892 = 28892